In [1]:
from scripts import download, extraction, utils
import os

In [2]:
search_query = "Entity Matching Record Linkage"
directory_path = f"experiments/articles/{search_query}"

In [3]:
# Download articles as HTML files
download.get_articles(search_query, directory_path)

https://arxiv.org/abs/2404.05622 doesn't redirect to the original link. Downloading...
Page downloaded: experiments/articles/Entity Matching Record Linkage/2404.05622.html
https://arxiv.org/abs/2404.05566 doesn't redirect to the original link. Downloading...
Page downloaded: experiments/articles/Entity Matching Record Linkage/2404.05566.html
https://arxiv.org/abs/2311.13923 doesn't redirect to the original link. Downloading...
Page downloaded: experiments/articles/Entity Matching Record Linkage/2311.13923.html
https://arxiv.org/abs/2212.05682 doesn't redirect to the original link. Downloading...
Page downloaded: experiments/articles/Entity Matching Record Linkage/2212.05682.html
https://arxiv.org/abs/2206.15089 doesn't redirect to the original link. Downloading...
Page downloaded: experiments/articles/Entity Matching Record Linkage/2206.15089.html
https://arxiv.org/abs/2201.04687 doesn't redirect to the original link. Downloading...
Page downloaded: experiments/articles/Entity Matching

In [4]:
def get_json_file_name(query: str) -> str:
    query = ' '.join(query.split())
    file_name = query.replace(' ', '_')
    file_name = file_name.lower() + '.json'
    return file_name


def extract_and_save_tables(articles_directory: str, save_path: str, file_name: str):
    articles_tables_map = extraction.extract_tables_from_directory(articles_directory)

    utils.check_path(save_path)
    file_name = os.path.join(save_path, file_name)

    num_tables = 0
    for article_id, article_tables in articles_tables_map.items():
        num_article_tables = len(article_tables)
        num_tables += num_article_tables
        print(f"Article ID: {article_id} - # Tables Found: {num_article_tables}")
    print(f"\nTotal number of tables found: {num_tables}")

    extraction.save_tables_to_json(articles_tables_map, file_name)

In [5]:
extract_and_save_tables(directory_path, f'experiments/extracted_tables', get_json_file_name(search_query))


# extract_and_save_tables('experiments/articles/cs_dataset', 'experiments/extracted_tables/cs_tables.json')

Article ID: 2007.05881 - # Tables Found: 6
Article ID: 1905.05337 - # Tables Found: 12
Article ID: 2206.15089 - # Tables Found: 0
Article ID: 2404.05566 - # Tables Found: 10
Article ID: 2110.14509 - # Tables Found: 9
Article ID: 2311.13923 - # Tables Found: 6
Article ID: 1212.5203 - # Tables Found: 0
Article ID: 1601.06630 - # Tables Found: 4
Article ID: 1402.0282 - # Tables Found: 2
Article ID: 2201.04687 - # Tables Found: 3
Article ID: 2404.05622 - # Tables Found: 1
Article ID: 1405.2048 - # Tables Found: 4
Article ID: 2212.05682 - # Tables Found: 0
Article ID: 1911.12930 - # Tables Found: 2
Article ID: 2112.03346 - # Tables Found: 5

Total number of tables found: 64
